### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2024
quarter = 3
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2025-01-01'

In [3]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [5]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 3) 
OR (year = 2024-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [7]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp

name  year  quarter     q_amt  Counter
0     3BBIF  8095       10   2443671        4
1       ACE  8095       10    750207        4
2    ADVANC  8095       10  32818977        4
4        AH  8095       10    764738        4
5       AIE  8095       10    138452        4
..      ...   ...      ...       ...      ...
219   WHAIR  8095       10    484373        4
220   WHART  8095       10   2247931        4
221   WHAUP  8095       10   1265063        4
222    WICE  8095       10    251253        4
223    WORK  8095       10    -36974        4

[209 rows x 5 columns]

In [9]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 3-1) 
OR (year = 2024-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [11]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.sample(5).style.format(format_dict)

In [13]:
dfp.name.unique().shape

(224,)

In [15]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(226, 15)

In [17]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(394, 9)

In [19]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.style.format(format_dict)

In [11]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2024 AND quarter = 'Q3'



146

In [21]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

name   id
120  GFPT  187
61   BSBM   79
281  SISB  683
214   PAP  342
381   VIH  611

In [23]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.style.format(format_dict)

In [14]:
rcds = df_ins.values.tolist()
len(rcds)

209

In [15]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [16]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [17]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].style.format(format_dict)

In [18]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].style.format(format_dict)

In [19]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].style.format(format_dict)

In [20]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].style.format(format_dict)

In [21]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).style.format(format_dict)

In [22]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).style.format(format_dict)

In [23]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).style.format(format_dict)

In [24]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



Empty DataFrame
Columns: [name, Nbr. of Group of 4 Quarters]
Index: []